In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

In [ ]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, I'm Sravan, a software engineer. Can you tell me about yourself?")])


In [ ]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, I'm Sravan, a software engineer. Can you tell me about yourself?"),
        AIMessage(content="Hello Sravan! I'm Gemma, a large language model trained by Groq. I can assist you with various tasks, including answering questions, providing information, and helping with programming-related queries."),
        HumanMessage(content="What's my name and what I do?")

    ]
)

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory() 
    return store[session_id]



In [ ]:
config = {"configurable":{"session_id":"chat_1"}}

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [ ]:
config = {"configurable":{"session_id":"chat_1"}}
with_message_history.invoke(
    [
        HumanMessage(content="Hi, I'm Sravan studying in Christ University, Bangalore"),
        AIMessage(content="Hi Sravan, nice meeting you. I've some friends studying in Christ University, Nitin and Sneha"),
        HumanMessage(content="Glad to hear that, how do you know them?")
     ],
     config=config
)

In [ ]:
# Difference between the session variables used

config_2 = {"configurable":{"session_id":"chat_1"}}
with_message_history.invoke(
    [
        HumanMessage(content="Hi, do you know anyone from Christ?")
     ],
     config=config_2
)

In [ ]:
config_2 = {"configurable":{"session_id":"chat_2"}}
with_message_history.invoke(
    [
        HumanMessage(content="Hi, do you know anyone from Christ?")
     ],
     config=config_2
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer the questions to the best of your ability in {language}"),
         MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model
response = chain.invoke({"messages":[HumanMessage(content="Hi, My name is Krish")],"language":"Hindi"})
response.content

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

config_3 = {"configurable": {"session_id":"chat3"}}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory() 
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages")

response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi, My name is Krish")],"language":"Tamil"},
    config=config_3
)
response.content

In [ ]:
config_4 = {"configurable": {"session_id":"chat3"}}
response2 = with_message_history.invoke(
    {"messages":[HumanMessage(content="What's my name")],"language":"Hindi"},
    config=config_4
)
response2.content

## Chatbots With Message History Implemented

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import  create_stuff_documents_chain 
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
import os
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import  create_stuff_documents_chain 
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")


os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

import bs4
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs= dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits = text_splitter.split_documents(docs)


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
splits

In [ ]:
vector_store = Chroma.from_documents(documents=splits,embedding=embeddings)
retriever = vector_store.as_retriever()

In [ ]:
retriever = vector_store.as_retriever()
retriever

In [ ]:
# Prompt Template

system_prompt = (
    """You are assistant for question-answering task
    Use the following pieces of retrieved context to answer the question. 
    If you dont know the answer say that you dont know. Use three sentences maximum and keep
    the sentence precise
    {context}
    """
)

# Langchain expects {context} variable inside system_prompt so that the retriever can pass the corresponding documents retrieved to this prompt as the parameter

prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("human","{input}")
])

#### RAG Pipeline

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain) 

In [ ]:
response = rag_chain.invoke({"input":"What is self reflection?"})

In [ ]:
response

In [ ]:
response["answer"]

In [ ]:
response2= rag_chain.invoke({"input":"How do we achieve it?"})
response2

## Adding Chat History

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [ ]:
contextualize_q_system_prompt = ("""
Given a chat history and the latest user question which might reference context in the chat history
formulate a standalone question which can be understood
without the chat history. Do not answer the question, just formulate it if needed and otherwise
return it as it is
""")
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [ ]:
from langchain_core.messages import AIMessage,HumanMessage
import os
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import  create_stuff_documents_chain 
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")


os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

import bs4
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs= dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits = text_splitter.split_documents(docs)


system_prompt = (
    """You are assistant for question-answering task
    Use the following pieces of retrieved context to answer the question. 
    If you dont know the answer say that you dont know. Use three sentences maximum and keep
    the sentence precise
    {context}
    """
)

contextualize_q_system_prompt = ("""
Given a chat history and the latest user question which might reference context in the chat history
formulate a standalone question which can be understood
without the chat history. Do not answer the question, just formulate it if needed and otherwise
return it as it is
""")


question_system_prompt = (
    """You are assistant for question-answering task
    Use the following pieces of retrieved context to answer the question. 
    If you dont know the answer say that you dont know. Use three sentences maximum and keep
    the sentence precise
    {context}
    """
)

# Langchain expects {context} variable inside system_prompt so that the retriever can pass the corresponding documents retrieved to this prompt as the parameter

question_prompt_template = ChatPromptTemplate.from_messages([
    ("system",question_system_prompt),
    ("human","{input}")
])

vector_store = Chroma.from_documents(documents=splits,embedding=embeddings)
retriever = vector_store.as_retriever()

contextualize_q_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

history_aware_retriever = create_history_aware_retriever(llm,retriever,contextualize_q_prompt)

question_answer_chain = create_stuff_documents_chain(llm,question_prompt_template)
rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

chat_history = []
question = "What is self reflection?"
response1= rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"]),
    ]
)

question2 = "Tell me more about it"
response2= rag_chain.invoke({"input":question2,"chat_history":chat_history})
response2["answer"]

In [ ]:
chat_history

In [ ]:
question2 = "Tell me more about it"
response2= rag_chain.invoke({"input":question2,"chat_history":chat_history})
response2

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

conversational_rag_chain.invoke(
    {
        "input":"What is Task Decomposition?"
    },
    config={"configurable":{"session_id":"abc123"}}
)['answer']

In [ ]:
conversational_rag_chain.invoke(
    {
        "input":"What are common ways of doing it?"
    },
    config={"configurable":{"session_id":"abc123"}}
)['answer']